In [ ]:
#librarys
import matplotlib.pyplot as plt

In [ ]:
#functions used by this script
%run ./ChiQuad.ipynb

# Name of the file to plot

In [ ]:
#call chiquadfunc
values = data("SN2004eo.txt")

## function returns

In [ ]:
#chiquad over degree of freedom
chiquaddof = values[0]

In [ ]:
#values fitted
fitvalues =  values[1]

In [ ]:
#non repeated bands
nonbands = values[2]

In [ ]:
#template functions
tpfun = values[3]

In [ ]:
#bands
bands = values[4]

In [ ]:
#data time, flux and flux error
xx = values[5]
zz = values[6]
ezz = values[7]

# Plot data def function

In [ ]:
#function that will return plot data according to band
def plot(band):

    #band index
    bandindex = [j for j, e in enumerate(nonbands) if e == band]
    
    #template list data plot
    x_plot = []
    z_plot = []
    for i in range(0,len(x_gr)):

        x_plot.append(x_gr[i])
        z_plot.append(fitvalues[1]*(tpfun[bandindex[0]][0](x_gr[i]) + fitvalues[2]*tpfun[bandindex[0]][1](x_gr[i]) + fitvalues[3]*tpfun[bandindex[0]][2](x_gr[i])))

    #real data lists plot
    x_plot_dat = []
    z_plot_dat = []
    z_error_plot_dat = []

    for i in range(0,len(xx)):

        if bands[i] == band:

            x_plot_dat.append(xx[i]-fitvalues[0])
            z_plot_dat.append(zz[i])
            z_error_plot_dat.append(ezz[i])
    
    #return template data points and real data plot points
    return x_plot, z_plot, x_plot_dat, z_plot_dat, z_error_plot_dat

# Choose a band to plot:

In [ ]:
dataplot = plot("cspg")

In [ ]:
%matplotlib notebook
plt.plot(dataplot[0], dataplot[1], 'k')
plt.errorbar(dataplot[2], dataplot[3], yerr=dataplot[4], fmt='.')
plt.xlim(-10,50)
plt.xlabel("tempo")
plt.ylabel("contagem/s")
plt.show()